 - "In computer science, genetic algorithms are a simulation of natural selection to solve computational challenges."
 
 - "A genetic algorithm includes a population (group) of individuals known as _chromosomes_. The chromosomes, each composed of genes that specify their traits, are competing to solve some problem. How well a chromosome solves a problem is defined by a _fitness function_.
 
 - "The genetic algorithm goes through _generations_. In each generation, the chromosomes that are more fit are more likely to be _selected_ to reproduce. There is also a probability in each generation that two chromosomes will have their genes merged. This is known as _crossover_. And finally, there is the important possibility in each generation that a gene in a chromosome may _mutate_ (randomly changed).
 
 - "After the fitness function of some individual in the population crosses some specified threshold, or the algorithm runs through some specified maximum of generations, the best individual (the one that scored highest in the fitness function) is returned."
 
 - Genetic algorithms depend on 3 partially or fully _stochastic_ operations: selection, crossover, and mutation.
 
 # A generic genetic algorithm
 
 The first step is to define an abstract class __Chromosome__. A chromosome must be able to do the following:
 1. Determine its own fitness.
 2. Create an instance with randomly selected genes (for the 1st generation).
 3. Implement crossover - combine with another of the same type to produce children.
 4. Mutate - make a small, fairly random change to its genes.
 
 Chromosome.py:

In [1]:
from __future__ import annotations
from typing import TypeVar, Tuple, Type
from abc import ABC, abstractmethod

T = TypeVar('T', bound = 'Chromosome') # for returning self
# This means that anything that fills in a variable that is of
# type T must be an instance of Chromosome of a subclass of
# Chromosome.


# Base class for all chromosomes; all methods must be overridden
class Chromosome(ABC):
    @abstractmethod
    def fitness(self) -> float:
        ...
    
    @classmethod
    @abstractmethod
    def random_instance(cls: Type[T]) -> T:
        ...
        
    @abstractmethod
    def crossover(self: T, other: T) -> Tuple[T, T]:
        ...
        
    @abstractmethod
    def mutate(self) -> None:
        ...

A genetic algorithm takes the following steps:
1. Create an initial population of random chromosomes for the first generation.
2. Measure the fitness of each chromosome in the current generation. If any exceeds the threshold, return it, and the algorithm ends.
3. Select some individuals to reproduce, with a higher probability of selecting those with the highest fitness.
4. Crossover(combine), with some probability, some of the selected chromosomes to create children that represent the population of the next generation.
5. Mutate, usually with a low probability, some of those chromosomes. The population of the new generation is now complete, and it replaces the population of the last generation.
6. Return to step 2 unless the maximum number of generations has been reached. In this case, return the best chromosome found so far.



Selection in step 3:
- Roulette-wheel selection (sometimes called fitness proportionate selection) gives every chromosome a chance of being picked, proportionate to its fitness.
- Tournament selection have a random set of chromosomes challenged against one another, and the one with the best fitness is selected.
- "One thing to keep in mind is that a higher number of participants in the tournament leads to less diversity in the population, because chromosomes with poor fitness are more likely to be eliminated in matchups."

genetic_algorithm.py:

In [ ]:
from __future__ import annotations
from typing import TypeVar, Generic, List, Tuple, Callable
from enum import Enum
from random import choices, random
from heapq import nlargest
from statistics import mean
'''from chromosome import Chromosome'''

C = TypeVar('C', bound = Chromosome) # type of the chromosome

class GeneticAlgorithm(Generic[C]):
    SelectionType = Enum('SelectionType', 'ROULETTE TOURNAMENT')
    
    def __init__(self, initial_population: List[C], threshold: float,
                max_generations: int = 100, mutation_chance: float = 0.01, 
                crossover_chance: float = 0.7, selection_type = SelectionType.TOURNAMENT) -> None:
        self._population: List[C] = initial_population
        self._threshold: float = threshold
        self._mutation_chance: float = mutation_chance
        self._crossover_chance: float = crossover_chance
        self._selection_type: GeneticAlgorithm.SelectionType = selection_type
        # the fitness key will be some callable that 
        # takes ._population[0] as argument with the fitness attribute
        # the ._fitness_key will be different for each of our subclasses
        self._fitness_key: Callable = type(self._population[0]).fitness
            
    # define the roulette method of selection
    # Use the probability distribution wheel to pick 2 parents
    # Note: will not work on negative fitness results
    # the choices function (from the Python module random) 
    ### takes a list of things we want to pick from, a weight list for those things, and no. of things to pick###
    # https://www.w3schools.com/python/ref_random_choices.asp
    def _pick_roulette(self, wheel: List[float]) -> Tuple[C, C]:
        return tuple(choices(self._population, weights = wheel, k = 2))
    
    # Choose num_participants at random and take the best 2
    def _pick_tournament(self, num_participants: int) -> Tuple[C, C]:
        participants: List[C] = choices(self._population, k = num_participants)
        # use the nlargest method from heapq to pick the 2 most fitting chromosomes
        return tuple(nlargest(2, participants, key = self._fitness_key))
    
    # Replace the population with a new generation of individuals
    def _reproduce_and_replace(self) -> None:
        new_population: List[C] = []
        
        # keep going until we've filled the new generation
        while len(new_population) < len(self._population):
            
            # pick the 2 parents
            if self._selection_type == GeneticAlgorithm.SelectionType.ROULETTE:
                # we are increasing the chance of more fit chromosomes so we use the fitnesses of 
                # them as our wheel/weights
                parents: Tuple[C, C] = self._pick_roulette([x.fitness for x in self._population])
                    
            else:
                # we will only take half if it is a tournament
                parents = self._pick_tournament(len(self._population) // 2)
            # potentially crossover the 2 parents
            if random() < self._crossover_chance:
                new_population.extend(parents[0].crossover(parents[1]))
            else:
                # if there are no children, the parents are just added to the new population
                new_population.extend(parents)
            
            # if we had an odd number, we'll have 1 extra , so we remove it
            if len(new_population) > len(self._population):
                new_population.pop()
                
            # replace reference
            self._population = new_population
            
    # With _mutation_chance probability mutate each individual
    def _mutate(self) -> None
    for individual in self._population:
        if random() < self._mutation_chance:
            individual.mutate()
    
    # The .run() method coordinates all the different steps and bring the population from one gen
    # to another. It also keeps track of the best chromosome found at any point in the search
    # Run the genetic algorithm for max_generations iterations
    # and return the best individual found
    
    def run(self) -> C:
        best: C = max(self._population, key = self._fitness_key)
        for generation in range(self._max_generations):
            # early exit if we hit threshold
            if best.fitness() >= self._threshold:
                return best
            print(f'Generation {generation} Best {best.fitness()} Avg
                  {mean(map(self._fitness_key, self._population))}')
            self._reproduce_and_replace()
            self.mutate()
            highest: C = max(self._population, key = self._fitness_key)
            if highest.fitness() > best.fitness()
                  best = highest # found a new best
        return best # best we found in _max_generations
                  